<a href="https://colab.research.google.com/github/kimhalyn/OPEN-API/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 라이브러리
# colab에 xmltodict 라이브러리가 기본 설치되지 않으므로 별도 설치(외부 라이브러리)
!pip install xmltodict 
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
from bs4 import BeautifulSoup
import pandas as pd
import requests, xmltodict, json

In [2]:
# 판다스로 csv 파일 불러오기_iloc 사용
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')

# 필요한 행,열만 추출
damcode = data.iloc[0:2,1]
#damcode = data.iloc[27,1]
for code in damcode:
  print(code)

2403201
2011602


In [3]:
# 우량 관측소 코드 조회(3차_새 코드)
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from time import sleep

url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/excll/list'
key = 'tMF2MGGUmsNNPf15Nr91%2BUA4YVQiWXVjLS7R0odxGSR0yfNjcHFkdRpCT6hXbILlBP1T%2BTnDJYz1kkDjiaIiQg=='  #디코딩 사용

# 댐코드 불러오기
data = pd.read_csv('/content/drive/MyDrive/water_resource_csv/damCode.csv')

# 필요한 코드 조회
damcode = data.iloc[0:2,1]

dam = [] 
for code in damcode:

  Params = '?' + urlencode({quote_plus('ServiceKey') : key,
                                quote_plus('damcode') : code})
  
  #print(Params)
  res = requests.get(url + Params).text.encode('utf-8') # 요청에 대한 응답내용(.encode빼기도 함)
  sleep(0.5)
  #print(res) 
  xmlobj = bs4.BeautifulSoup(res, 'lxml') #lxml 파서(구문분석)  'lxml-xml'

  #item 태그 분리
  rows = xmlobj.findAll('item') # 매치되는 문자열을 리스트로 리턴
  #print(rows)

  # 반복문
  rowList = []
  nameList = []
  columnList = []

  rowsLen = len(rows)
  
  for i in range(0, rowsLen):

    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):   

      if i == 0:
        nameList.append(columns[j].name) # 칼럼명
      eachColumn = columns[j].text # 칼럼값
      columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []

  result = pd.DataFrame(rowList, columns=nameList)
  dam.append(result)




# csv파일로 저장
# df = pd.DataFrame(rowList, columns=nameList)
# df.to_csv("review.csv", encoding='utf-8-sig')

In [ ]:
print()